![Python_logo](https://www.python.org/static/community_logos/python-logo-master-v3-TM.png)


 # **Cortex Game: Round2--Conditional Amount**

Please note that you need to run this notebook 'Round2--Conditional Amount' first, before running the notebook 'Round2--Probability of Giving'.   

> Before playing the game, you need to connect to SASPy first.
>
>> If it is your first time, please follow the 4 steps mentioned below!

***
## **Connect to SASPy**

**0- Connect to your Google Drive folder**

In [ ]:
my_folder = "/content/drive/MyDrive/retoSas"

from google.colab import drive
drive.mount('/content/drive')

# Change the following code to set your Drive folder
import os
os.chdir(my_folder)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/retoSas


**1- Make sure that your Python version is 3.3 or higher as well as your Java version is 1.8.0_162 or higher**

In [ ]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [ ]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [ ]:
pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 2.0 MB/s 
  Created wheel for saspy: filename=saspy-4.4.1-py3-none-any.whl size=9937304 sha256=bc4d00a72e08d53bc3741c44aa70e4be394efe13efb65719a2fab7a3aa2f2bcc
  Stored in directory: /root/.cache/pip/wheels/0a/f6/ab/f7173c8a824856c66f6fd50194bb8fd3ef93fb4f1824be8d0c
Successfully built saspy


**3- Create the configuration file "sascfg_personal.py"**
Please, check that your Home Region is correct, you can check it at [ODA-SAS](https://welcome.oda.sas.com/home)

In [ ]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 1
'iomhost' : ['odaws01-usw2.oda.sas.com','odaws02-usw2.oda.sas.com','odaws03-usw2.oda.sas.com','odaws04-usw2.oda.sas.com'],
#US Home Region 2
#'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
#European Home Region 1
#'iomhost' : ['odaws01-euw1.oda.sas.com','odaws02-euw1.oda.sas.com'],
#Asia Pacific Home Region 1
#'iomhost' : ['odaws01-apse1.oda.sas.com','odaws02-apse1.oda.sas.com'],
#Asia Pacific Home Region 2
#'iomhost' : ['odaws01-apse1-2.oda.sas.com','odaws02-apse1-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4- Create your .authinfo**

If there is no .authinfo file, you can create this

In [ ]:
#%%writefile .authinfo
#oda user USR password PSW

Copy this file to home

In [ ]:
!cp .authinfo ~/.authinfo

**5- Establish Connection (Need to do this step each time you use SASPy)**

In [ ]:
import saspy
sas_session = saspy.SASsession(cfgfile=os.path.join(
    my_folder,"sascfg_personal.py"))
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 356



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/retoSas/sascfg_personal.py
WORK Path             = /saswork/SAS_work2D310000CE07_odaws01-usw2.oda.sas.com/SAS_work69B50000CE07_odaws01-usw2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 52743


***
## Connect to Cortex Data Sets

Load Cortex datasets from SAS Studio

In [ ]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


5                                                          The SAS System                      Friday, December  2, 2022 05:57:00 PM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         
27             libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
28         
29         
30         
31         ods html5 (id=saspy_internal) close;ods listing;
32         

6                                                          The SAS System                      Friday, December  2, 2022 05:57:00 PM

33         


For local Jupyter

In [ ]:
#%%SAS sas_session
#libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';

### Transform cloud SAS dataset to Python dataframe (pandas)

> For reference: 

> 1- [Pandas library](https://pandas.pydata.org/docs/user_guide/index.html)

> 2- [sklearn.model_selection for data partition](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

## Merge the Data

In [ ]:
#Step1 Merge the Data
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
856747,2856748.0,KEELER,BETH,1.0,57.0,700.0,High School,Rural,8.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,50.0
791407,2791408.0,COFFEE,RUBY,1.0,21.0,49200.0,University / College,Suburban,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,10.0


In [ ]:
data_merge.corr()

,ID,Woman,Age,Salary,SeniorList,NbActivities,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
ID,1.000000,-0.004276,-0.008712,-0.004636,0.003979,0.005697,0.001076,-0.003654,0.003394,0.000229,-0.000453,-0.005094,-0.000410,0.000409,-0.001091,-0.003065,NaN,-0.000189
Woman,-0.004276,1.000000,0.032079,-0.054439,0.001698,-0.004229,-0.007491,0.009333,-0.005368,0.003854,0.064978,0.032388,0.063812,-0.011307,0.016610,0.095114,NaN,0.043386
Age,-0.008712,0.032079,1.000000,0.031511,-0.002698,0.011257,0.008921,-0.005146,0.009085,0.001943,-0.054877,-0.030579,-0.054786,0.011648,-0.016480,-0.094376,NaN,-0.038916
Salary,-0.004636,-0.054439,0.031511,1.000000,-0.006253,0.002903,0.004151,-0.012498,0.008911,-0.002366,0.088754,0.045249,0.085745,0.009253,0.031044,-0.039594,NaN,0.067602
SeniorList,0.003979,0.001698,-0.002698,-0.006253,1.000000,0.524657,0.488871,0.382062,0.380335,0.736569,0.084113,-0.061878,0.047113,0.006976,0.003274,-0.087352,NaN,0.004176
NbActivities,0.005697,-0.004229,0.011257,0.002903,0.524657,1.000000,0.590529,-0.051716,0.537397,0.421262,0.129311,-0.056506,0.077939,0.175988,0.045125,-0.129571,NaN,0.016713
Referrals,0.001076,-0.007491,0.008921,0.004151,0.488871,0.590529,1.000000,-0.046707,0.486401,0.382357,0.115190,-0.051738,0.069581,0.162628,0.039388,-0.121569,NaN,0.014993
Recency,-0.003654,0.009333,-0.005146,-0.012498,0.382062,-0.051716,-0.046707,1.000000,-0.287239,0.455561,-0.079105,0.030040,-0.052566,-0.100841,-0.017540,0.116075,NaN,-0.007751
Frequency,0.003394,-0.005368,0.009085,0.008911,0.380335,0.537397,0.486401,-0.287239,1.000000,0.583472,0.245425,-0.114186,0.148379,0.162437,0.034463,-0.078655,NaN,0.014064
Seniority,0.000229,0.003854,0.001943,-0.002366,0.736569,0.421262,0.382357,0.455561,0.583472,1.000000,0.136465,-0.084357,0.080835,0.037112,0.013494,0.011291,NaN,0.008663


## Treating Missing Values

>Please be aware that deleting all missing values can induce a selection bias. 
Some missing values are very informative. For example, when MinGift is missing, it means that the donor never gave in the past 10 years (leading to but excluding last year). Instead of deleting this information, replacing it by 0 is more appropriate!

> A good understanding of the business case and the data can help you come up with more appropriate strategies to deal with missing values.

In [ ]:
# In this case, we are replacing MinGift by 0.
# You can do the same for what you think is reasonable for dealing with the other variables.

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=0)  
# using k nearest for frequency 

from sklearn.impute import KNNImputer
#knn_imputer = KNNImputer(missing_values=np.nan, n_neighbors=5, weights='uniform', metric='nan_euclidean')

# define imputer
#imputer = KNNImputer(n_neighbors=5, metric='nan_euclidean')
#X = data_merge[['Frequency']] 
# fit on the dataset
# transform the dataset
#Xtrans = imputer.fit_transform(X)
# print total missing
#X_r = pd.DataFrame(Xtrans)



/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
#data_merge[["Frequency_test"]] = Xtrans.tolist()

## Data Partition

In [ ]:
# The code below is an illustration on how to sample data on train and validation samples.
# You could use another library or a built-in function to perform sampling.

from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.4,random_state=5678) # you can change the percentage
train.sample(5)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
58677,2058678.0,SHINE,BRENDA,1.0,56.0,57200.0,University / College,City,7.0,1.0,...,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,1.0,25.0
470187,2470188.0,TRAVIS,DUANE,0.0,52.0,223800.0,High School,City,2.0,0.0,...,1.0,1.0,10.0,10.0,10.0,0.0,0.0,0.0,1.0,20.0
201440,2201441.0,JONES,MAUREEN,1.0,19.0,19300.0,High School,Rural,0.0,0.0,...,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,1.0,10.0
172693,2172694.0,HANKINSON,DELORIS,1.0,18.0,7300.0,University / College,Suburban,5.0,0.0,...,1.0,4.0,40.0,40.0,40.0,0.0,0.0,0.0,1.0,10.0
221146,2221147.0,FOSS,JENNIFER,1.0,70.0,78600.0,University / College,Rural,9.0,3.0,...,2.0,8.0,30.0,10.0,20.0,0.0,0.0,0.0,1.0,30.0



## Prebuilt Models

***
### Linear Regression Model


> The [sk-learn library]( https://scikit-learn.org/stable/index.html) offers more advanced models. 

In [ ]:
from sklearn import linear_model

#comment: it's numpy array
X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict=regr.predict(X_valid)

print(regr_predict)

[ 53.68488365  12.67643999 123.05966359 ...  51.1950202   38.27147761
  63.34047567]


In [ ]:
#you can change the criteria

import numpy as np
from sklearn import metrics
#MAE
print(metrics.mean_absolute_error(Y_valid,regr_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,regr_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,regr_predict)))

64.07638086724123
53185.57275158189
230.6199747454281


## Regression Tree Model（Py）

In [ ]:
from sklearn.tree import DecisionTreeRegressor

X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_valid = validation['AmtThisYear']

DT_model = DecisionTreeRegressor(max_depth=4, random_state=0).fit(X_train,Y_train)
DT_predict = DT_model.predict(X_valid) #Predictions on Testing data
print(DT_predict)

[ 52.05412054  32.38836364 171.9390357  ...  44.18701059  55.65287539
  44.18701059]


In [ ]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid,DT_predict))
#MSE
print(metrics.mean_squared_error(Y_valid,DT_predict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,DT_predict)))

63.344317055007764
52706.63600925966
229.57925866519315


### **Other models may also be helpful for this game**

Reference: https://scikit-learn.org/stable/supervised_learning.html


# GRADIENT BOOST regressor
Add specs of library from histgradient

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
X_train = train[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact','MinGift', 'GaveLastYear','AmtLastYear','Woman', 'NbActivities']] 
Y_valid = validation['AmtThisYear']

gdBoostReg = HistGradientBoostingRegressor(max_depth=5, random_state=12345, loss='poisson').fit(X_train, Y_train)
gdBoostPredict = gdBoostReg.predict(X_valid)
print(gdBoostPredict)

[ 53.78431132  34.64793902 162.18892188 ...  47.2019618   53.75137752
  63.22023268]


In [ ]:
#you can change the criteria
#MAE
print(metrics.mean_absolute_error(Y_valid, gdBoostPredict))
#MSE
print(metrics.mean_squared_error(Y_valid, gdBoostPredict))
#RMSE
print(np.sqrt(metrics.mean_squared_error(Y_valid,gdBoostPredict)))

63.56281274154598
52671.71995919418
229.50320250313322


## Scoring New Data

### Prepare data for scoring

In [ ]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

 ### Score new data based on your champion model
 
 Pick your champion model from previous steps and use it to predict next year donations. 
 
 In this case, the linear regression model performed better than the regression tree based on the MSE criteria.

### Predict 'amount given' for members who were contacted

In [ ]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_contact[['MinGift']] = scoring_data_contact[['MinGift']].fillna(value=0) 

#scoring_data_contact[['Frequency']] = scoring_data_contact[['Frequency']].fillna(value= data_merge[['Frequency_test']].mean()) 
#scoring_data_contact.head()

X = scoring_data_contact[['Age', 'Salary','Contact','MinGift', 'AmtLastYear','GaveLastYear','Woman', 'NbActivities']] 

regr_predict_contact= gdBoostReg.predict(X)

scoring_data_contact['Prediction'] = regr_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,ID,AmtContact
0,2000001.0,62.010280
1,2000002.0,57.850230
2,2000003.0,88.794168
3,2000004.0,44.371728
4,2000005.0,92.198468


### Predict 'amount given' for members who were not contacted

In [ ]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

# Perform the same strategy for handling missing values for the score dataset.
# In this case, we will only replace missing values of the MinGift variable.

scoring_data_nocontact[['MinGift']] = scoring_data_nocontact[['MinGift']].fillna(value=0)
#scoring_data_nocontact[['Frequency']] = scoring_data_nocontact[['Frequency']].fillna(value= data_merge[['Frequency_test']].mean())
#scoring_data_nocontact.head()

X = scoring_data_nocontact[['Age', 'Salary','Contact','MinGift','AmtLastYear', 'GaveLastYear','Woman', 'NbActivities' ]] 

regr_predict_nocontact=gdBoostReg.predict(X)

scoring_data_nocontact['Prediction'] = regr_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


,ID,AmtNoContact
0,2000001.0,65.689215
1,2000002.0,57.850230
2,2000003.0,96.871003
3,2000004.0,44.683829
4,2000005.0,109.107609


In [ ]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,62.010280,65.689215
1,2000002.0,57.850230,57.850230
2,2000003.0,88.794168,96.871003


## Exporting Results to a CSV File

In [ ]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [ ]:
# Reminder: You are now done with step 1 of Round 2 on predicting the conditional amount.
# Next, to complete Round2, you need to perform step 2 to predict the probability of giving, calculate the uplift and prepare your decision.

In [ ]:
!head Round2_Output_amt.csv

ID,AmtContact,AmtNoContact
2000001.0,62.01027987812166,65.68921525819002
2000002.0,57.850230493681686,57.850230493681686
2000003.0,88.7941682956768,96.87100330689618
2000004.0,44.37172755198852,44.683829101762385
2000005.0,92.19846768650967,109.10760912372105
2000006.0,35.241334961688665,35.20956479085273
2000007.0,30.580994182556633,30.553425323131513
2000008.0,67.77956300927647,71.80077744163349
2000009.0,43.040631717143114,43.040631717143114
